In [1]:
import numpy as np


In [2]:
def tanh(x):
   
    return np.tanh(x);

def tanh_prime(x,o_r):    
    
    y= 1-np.tanh(x)**2;
    return y*o_r

In [3]:

def cal_err(target, actual):
  
    return np.mean(np.power(target-actual, 2));


def cald_err_der(target, actual):
    return 2*(actual-target)/target.size;

In [7]:
class layer:
    def __init__(self):
        self.input=None
        self.output=None
    def forwardpass(inputs,weight):
        raise NotImplementedError
    def backpass(output,errorrate,learnrate):
        raise NotImplementedError

In [62]:
class Fclayer(layer):
    def __init__(self,input_dim,output_dim,B1=0.9,B2=0.99):
        self.Weight=np.random.rand(input_dim, output_dim) - 0.5
        self.bias=np.random.rand(1, output_dim) - 0.5
        self.Vdw=np.zeros((input_dim,output_dim))
        self.Sdw=np.zeros((input_dim,output_dim))
        self.Vdb=np.zeros((1,output_dim))
        self.Sdb=np.zeros((1,output_dim))
        
        self.B1=B1
        self.B2=B2
        self.eps=0.0001
    def forward(self,data):# data shape is batchsize,....,hidden layer
        self.input=data
        output=np.dot(data,self.Weight)+self.bias
        return output
    def zero_grad(self):
        
        self.Vdw*=0
        self.Sdw*=0
        self.Vdb*=0
        self.Sdb*=0
        
    def update(self,lr):
        
        self.Weight=self.Weight-(lr*self.Vdw)/(np.sqrt(self.Sdw))
        self.bias=self.bias-(lr*self.Vdb)/(np.sqrt(self.Sdb))
        
        
    def backward(self,err,lr):
        der_W=np.dot((self.input).reshape(-1,self.input.shape[-1]).T,err.reshape(-1,err.shape[-1])).astype('float32') 
        der_inp=np.dot(err,self.Weight.T)
        der_B=err
        
        self.Vdw=self.B1*(self.Vdw)+(1-self.B1)*der_W
        self.Sdw=self.B2*(self.Sdw)+(1-self.B2)*(der_W**2+self.eps)# a small constant is added when derivative becomes zero numerator becomes infinits and loss becomes nan
        self.Vdb=(1-self.B1)*(self.Vdb)+self.B1*der_B
        self.Sdb=(1-self.B2)*(self.Sdb)+self.B2*(der_B**2+self.eps)
        
        self.update(lr)
        return der_inp


In [63]:
class ActivationLayer(layer):
    def __init__(self, activation):
        if(activation=='tanh'):
            self.activation = tanh
            self.activation_prime = tanh_prime
        if(activation=='relu'):
            self.activation = relu
            self.activation_prime = relu_prime
        if(activation=='sigmoid'):
            self.activation = sigmoid
            self.activation_prime = sigmoid_prime
       

    # returns the activated input
    def forward(self, input_data):
        self.input = input_data
        output = self.activation(self.input)
        return output
    def zero_grad(self):
        pass
    # Returns input_error=dE/dX for a given output_error=dE/dY.
 
    def backward(self, output_error, learning_rate):
        return self.activation_prime(self.input,output_error) 

In [74]:
class network:
    def __init__(self):
        self.layers=[]
        self.loss=None
        self.loss_prime=None
    def add(self,layer):
        self.layers.append(layer)
    def use(self,loss,loss_prime):
        self.loss=loss
        self.error=loss_prime
    def fit(self,traindata,trainlabel,epochs,learning_rate):
        print('fit')
        no_sample=len(traindata)
       
        for i in range (0,epochs):
            print(epochs-i)
            err=0
            for j in range(no_sample):
                layer_output=traindata[j]
                for each_layer in self.layers:
                    layer_output=each_layer.forward(layer_output)
                err += self.loss(trainlabel[j], layer_output)             
                error = self.error(trainlabel[j], layer_output)
                for each_layer in reversed(self.layers):
                    error=each_layer.backward(error,learning_rate)
            print("epoch :  ",i, "loss  :",err)
            for each_layer in self.layers:
                    layer_output=each_layer.zero_grad()
               
               
                
                                            
        
    def predict(self,testdata):
        print('predict')
        no_sample=len(testdata)
        result=[]
        for i in range(0, no_sample):            
           
                layer_output=testdata[i]
                for each_layer in self.layers:
                    layer_output=each_layer.forward(layer_output)
                result.append(layer_output,)
        return result

In [77]:
activ='tanh'
net = network()
net.add(Fclayer(2, 3))
net.add(ActivationLayer(activ ))
net.add(Fclayer(3, 3))
net.add(ActivationLayer(activ))
net.add(Fclayer(3, 1))
net.add(ActivationLayer(activ))
x_train = np.array([[[0,0]], [[0,1]], [[1,0]], [[1,1]]])
y_train = np.array([[0], [1], [1], [0]])
# train
net.use(cal_err, cald_err_der)
#Smaller learning rates require more training epochs given the smaller changes made to the weights each update, whereas larger learning rates result in rapid changes and require fewer training epochs. ... If you have time to tune only one hyperparameter, tune the learning rate
net.fit(x_train, y_train, 10000, learning_rate=0.01)

fit
10000
epoch :   0 loss  : 1.867060211792705
9999
epoch :   1 loss  : 1.6698029408029385
9998
epoch :   2 loss  : 1.5422609122836
9997
epoch :   3 loss  : 1.4432972938713176
9996
epoch :   4 loss  : 1.3632842078009195
9995
epoch :   5 loss  : 1.298468217665276
9994
epoch :   6 loss  : 1.2464307764450766
9993
epoch :   7 loss  : 1.2050846762906693
9992
epoch :   8 loss  : 1.17257127535041
9991
epoch :   9 loss  : 1.1472802633558277
9990
epoch :   10 loss  : 1.1277985203808385
9989
epoch :   11 loss  : 1.1128485684076614
9988
epoch :   12 loss  : 1.1013603441244446
9987
epoch :   13 loss  : 1.0924990214212245
9986
epoch :   14 loss  : 1.0856213922208695
9985
epoch :   15 loss  : 1.0802282179793878
9984
epoch :   16 loss  : 1.0759250086747347
9983
epoch :   17 loss  : 1.0723903406512294
9982
epoch :   18 loss  : 1.069351679595583
9981
epoch :   19 loss  : 1.066568851162561
9980
epoch :   20 loss  : 1.063824541898501
9979
epoch :   21 loss  : 1.0609205041004828
9978
epoch :   22 loss  :

9743
epoch :   257 loss  : 0.9409974471979201
9742
epoch :   258 loss  : 0.9504966300169185
9741
epoch :   259 loss  : 0.9406117327474994
9740
epoch :   260 loss  : 0.9500993774507508
9739
epoch :   261 loss  : 0.9402216220272255
9738
epoch :   262 loss  : 0.9496931931566074
9737
epoch :   263 loss  : 0.9398273280315985
9736
epoch :   264 loss  : 0.9492782492070568
9735
epoch :   265 loss  : 0.9394290357245676
9734
epoch :   266 loss  : 0.9488547030691192
9733
epoch :   267 loss  : 0.9390269041448278
9732
epoch :   268 loss  : 0.9484226991889451
9731
epoch :   269 loss  : 0.938621065756328
9730
epoch :   270 loss  : 0.9479823673650015
9729
epoch :   271 loss  : 0.9382116268181376
9728
epoch :   272 loss  : 0.9475338239885911
9727
epoch :   273 loss  : 0.9377986668127131
9726
epoch :   274 loss  : 0.9470771714108334
9725
epoch :   275 loss  : 0.9373822398309207
9724
epoch :   276 loss  : 0.9466125002387991
9723
epoch :   277 loss  : 0.9369623748753415
9722
epoch :   278 loss  : 0.946139

epoch :   522 loss  : 0.4555530370082269
9477
epoch :   523 loss  : 0.5422369225293568
9476
epoch :   524 loss  : 0.4490874448226337
9475
epoch :   525 loss  : 0.5367970762429837
9474
epoch :   526 loss  : 0.4427265675218203
9473
epoch :   527 loss  : 0.5314485340159354
9472
epoch :   528 loss  : 0.4363928591871062
9471
epoch :   529 loss  : 0.5261563675617142
9470
epoch :   530 loss  : 0.42989824707333535
9469
epoch :   531 loss  : 0.5207695812720062
9468
epoch :   532 loss  : 0.42289558045869236
9467
epoch :   533 loss  : 0.5149910507265679
9466
epoch :   534 loss  : 0.4151075540590121
9465
epoch :   535 loss  : 0.5086445601137448
9464
epoch :   536 loss  : 0.4064047674413843
9463
epoch :   537 loss  : 0.5016301336691034
9462
epoch :   538 loss  : 0.3968161102718005
9461
epoch :   539 loss  : 0.4938907031494397
9460
epoch :   540 loss  : 0.3865399001606338
9459
epoch :   541 loss  : 0.4854862299290349
9458
epoch :   542 loss  : 0.3757975677483498
9457
epoch :   543 loss  : 0.47653351

epoch :   785 loss  : 0.8601131798610077
9214
epoch :   786 loss  : 0.8998240016066028
9213
epoch :   787 loss  : 0.861729134207359
9212
epoch :   788 loss  : 0.9018923220178164
9211
epoch :   789 loss  : 0.8632498437398942
9210
epoch :   790 loss  : 0.9038487604601511
9209
epoch :   791 loss  : 0.8647037873041804
9208
epoch :   792 loss  : 0.9057300254745043
9207
epoch :   793 loss  : 0.8661233837192498
9206
epoch :   794 loss  : 0.9075778342688033
9205
epoch :   795 loss  : 0.8675444091458103
9204
epoch :   796 loss  : 0.9094369584525904
9203
epoch :   797 loss  : 0.8690041978011296
9202
epoch :   798 loss  : 0.91135026648615
9201
epoch :   799 loss  : 0.8705384460256474
9200
epoch :   800 loss  : 0.9133491553181189
9199
epoch :   801 loss  : 0.8721777167325196
9198
epoch :   802 loss  : 0.9154395369645945
9197
epoch :   803 loss  : 0.8739460454805064
9196
epoch :   804 loss  : 0.9176000154953681
9195
epoch :   805 loss  : 0.8758596098418991
9194
epoch :   806 loss  : 0.9198137774370

epoch :   1048 loss  : 0.7176466771457227
8951
epoch :   1049 loss  : 0.7645427430005548
8950
epoch :   1050 loss  : 0.7117924834043072
8949
epoch :   1051 loss  : 0.7600417957958443
8948
epoch :   1052 loss  : 0.7058285330435211
8947
epoch :   1053 loss  : 0.7554644000550697
8946
epoch :   1054 loss  : 0.6997566365096038
8945
epoch :   1055 loss  : 0.7508124305240695
8944
epoch :   1056 loss  : 0.6935785649711981
8943
epoch :   1057 loss  : 0.7460877340637296
8942
epoch :   1058 loss  : 0.6872960452893517
8941
epoch :   1059 loss  : 0.7412921341118607
8940
epoch :   1060 loss  : 0.6809107887135172
8939
epoch :   1061 loss  : 0.7364274478997803
8938
epoch :   1062 loss  : 0.6744245080701857
8937
epoch :   1063 loss  : 0.731495530187623
8936
epoch :   1064 loss  : 0.6678390290752632
8935
epoch :   1065 loss  : 0.7264983542225668
8934
epoch :   1066 loss  : 0.6611563917025045
8933
epoch :   1067 loss  : 0.7214381214625893
8932
epoch :   1068 loss  : 0.6543790188663603
8931
epoch :   1069

...

epoch :   8700 loss  : 0.005028151325093564
1299
epoch :   8701 loss  : 0.0048187060788235534
1298
epoch :   8702 loss  : 0.005027825269262777
1297
epoch :   8703 loss  : 0.0048186081747009285
1296
epoch :   8704 loss  : 0.005027500318609118
1295
epoch :   8705 loss  : 0.004818510907770154
1294
epoch :   8706 loss  : 0.005027174917084305
1293
epoch :   8707 loss  : 0.0048184136141399695
1292
epoch :   8708 loss  : 0.005026850394615681
1291
epoch :   8709 loss  : 0.00481831655702819
1290
epoch :   8710 loss  : 0.005026526833139865
1289
epoch :   8711 loss  : 0.0048182200533140535
1288
epoch :   8712 loss  : 0.005026203620147932
1287
epoch :   8713 loss  : 0.004818123996144435
1286
epoch :   8714 loss  : 0.005025880719186644
1285
epoch :   8715 loss  : 0.004818027909156532
1284
epoch :   8716 loss  : 0.00502555878087183
1283
epoch :   8717 loss  : 0.0048179328039295215
1282
epoch :   8718 loss  : 0.005025237157760954
1281
epoch :   8719 loss  : 0.004817837486127333
1280
epoch :   8720 lo

epoch :   8967 loss  : 0.004808250843844222
1032
epoch :   8968 loss  : 0.004989185194272274
1031
epoch :   8969 loss  : 0.004808188762616687
1030
epoch :   8970 loss  : 0.004988927409685393
1029
epoch :   8971 loss  : 0.004808126868959394
1028
epoch :   8972 loss  : 0.004988669606726767
1027
epoch :   8973 loss  : 0.004808065445017701
1026
epoch :   8974 loss  : 0.004988412363632429
1025
epoch :   8975 loss  : 0.004808003889962154
1024
epoch :   8976 loss  : 0.004988155519627952
1023
epoch :   8977 loss  : 0.004807942672004377
1022
epoch :   8978 loss  : 0.0049878992574907376
1021
epoch :   8979 loss  : 0.004807881679458413
1020
epoch :   8980 loss  : 0.004987643257367096
1019
epoch :   8981 loss  : 0.004807820877704632
1018
epoch :   8982 loss  : 0.004987387604770201
1017
epoch :   8983 loss  : 0.004807760536934806
1016
epoch :   8984 loss  : 0.004987132413854068
1015
epoch :   8985 loss  : 0.0048076998891914075
1014
epoch :   8986 loss  : 0.0049868778362892816
1013
epoch :   8987 lo

epoch :   9234 loss  : 0.00495837831034001
765
epoch :   9235 loss  : 0.004801520016457474
764
epoch :   9236 loss  : 0.004958171090794581
763
epoch :   9237 loss  : 0.0048014801610076294
762
epoch :   9238 loss  : 0.004957964687124913
761
epoch :   9239 loss  : 0.004801440083804369
760
epoch :   9240 loss  : 0.00495775821463279
759
epoch :   9241 loss  : 0.004801400467948937
758
epoch :   9242 loss  : 0.004957552323739317
757
epoch :   9243 loss  : 0.004801360667623128
756
epoch :   9244 loss  : 0.004957346371414309
755
epoch :   9245 loss  : 0.004801321051592137
754
epoch :   9246 loss  : 0.0049571412332749636
753
epoch :   9247 loss  : 0.004801281980487061
752
epoch :   9248 loss  : 0.004956936233539686
751
epoch :   9249 loss  : 0.004801242713911443
750
epoch :   9250 loss  : 0.0049567314124623185
749
epoch :   9251 loss  : 0.004801203385253306
748
epoch :   9252 loss  : 0.004956526783392543
747
epoch :   9253 loss  : 0.004801164254989051
746
epoch :   9254 loss  : 0.00495632275530

epoch :   9495 loss  : 0.004797238149848934
504
epoch :   9496 loss  : 0.0049338746287280605
503
epoch :   9497 loss  : 0.00479721132086899
502
epoch :   9498 loss  : 0.004933706004055735
501
epoch :   9499 loss  : 0.0047971848409241805
500
epoch :   9500 loss  : 0.00493353792445653
499
epoch :   9501 loss  : 0.00479715816663355
498
epoch :   9502 loss  : 0.004933370103899727
497
epoch :   9503 loss  : 0.004797131686050566
496
epoch :   9504 loss  : 0.0049332024834077195
495
epoch :   9505 loss  : 0.004797104992119975
494
epoch :   9506 loss  : 0.0049330350927724
493
epoch :   9507 loss  : 0.004797078949996665
492
epoch :   9508 loss  : 0.004932868177780711
491
epoch :   9509 loss  : 0.004797052851374548
490
epoch :   9510 loss  : 0.004932700992804376
489
epoch :   9511 loss  : 0.0047970266630012835
488
epoch :   9512 loss  : 0.004932534531911864
487
epoch :   9513 loss  : 0.004797000420799994
486
epoch :   9514 loss  : 0.004932368121751519
485
epoch :   9515 loss  : 0.0047969741424994

242
epoch :   9758 loss  : 0.004913843110750839
241
epoch :   9759 loss  : 0.004794310896074948
240
epoch :   9760 loss  : 0.004913704844736487
239
epoch :   9761 loss  : 0.00479429277023611
238
epoch :   9762 loss  : 0.004913566432483864
237
epoch :   9763 loss  : 0.004794274754757874
236
epoch :   9764 loss  : 0.004913428267587854
235
epoch :   9765 loss  : 0.004794256378289966
234
epoch :   9766 loss  : 0.004913290842510488
233
epoch :   9767 loss  : 0.004794238592784209
232
epoch :   9768 loss  : 0.004913152985928994
231
epoch :   9769 loss  : 0.0047942206301971775
230
epoch :   9770 loss  : 0.004913015722089796
229
epoch :   9771 loss  : 0.00479420247949547
228
epoch :   9772 loss  : 0.00491287872745528
227
epoch :   9773 loss  : 0.004794184724582115
226
epoch :   9774 loss  : 0.004912741422101008
225
epoch :   9775 loss  : 0.004794166859264794
224
epoch :   9776 loss  : 0.0049126046775160555
223
epoch :   9777 loss  : 0.00479414925127203
222
epoch :   9778 loss  : 0.0049124683362

In [78]:
out = net.predict(x_train)
for k in out:
  for i in k:
    
   
    for j in i:
        print(np.power(round(j,0),2))

predict
0.0
1.0
1.0
0.0
